In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import  GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor


In [2]:
# import your data
data = pd.read_csv('mock_data.csv')

In [7]:
data.head()

,student_id,student_name,c_internal_exam1_marks,c_internal_exam2_marks,c_attendance_percentage,c_final_exam_marks,c_preparation_time_daily_hours,java_internal_exam1_marks,java_internal_exam2_marks,java_attendance_percentage,...,cn_internal_exam1_marks,cn_internal_exam2_marks,cn_attendance_percentage,cn_final_exam_marks,cn_preparation_time_daily_hours,dsa_internal_exam1_marks,dsa_internal_exam2_marks,dsa_attendance_percentage,dsa_final_exam_marks,dsa_preparation_time_daily_hours
0,1,Marcel Rack,2,18,23.46,14,11.68,35,29,34.43,...,33,38,78.81,25,3.37,11,38,21.86,88,16.76
1,2,Barrett Pitfield,26,25,6.42,84,7.94,31,21,2.45,...,2,49,26.49,1,12.34,0,28,77.70,91,13.11
2,3,Marley Perin,17,32,55.67,29,16.15,6,20,54.37,...,50,48,52.05,58,18.47,32,8,11.64,89,23.69
3,4,Dewain McPhelim,7,11,73.16,20,9.23,28,30,69.54,...,25,39,62.62,50,19.42,16,14,86.84,70,22.30
4,5,Maryjo Grundy,45,6,23.29,97,10.80,1,1,37.48,...,22,8,91.70,61,10.92,25,33,62.14,95,8.54


In [8]:
data.describe()

,student_id,c_internal_exam1_marks,c_internal_exam2_marks,c_attendance_percentage,c_final_exam_marks,c_preparation_time_daily_hours,java_internal_exam1_marks,java_internal_exam2_marks,java_attendance_percentage,java_final_exam_marks,...,cn_internal_exam1_marks,cn_internal_exam2_marks,cn_attendance_percentage,cn_final_exam_marks,cn_preparation_time_daily_hours,dsa_internal_exam1_marks,dsa_internal_exam2_marks,dsa_attendance_percentage,dsa_final_exam_marks,dsa_preparation_time_daily_hours
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,...,60.000000,60.000000,60.00000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,30.500000,24.383333,24.800000,46.862333,54.216667,13.377667,21.300000,25.066667,47.344833,50.616667,...,24.950000,23.633333,42.97200,51.733333,11.647500,25.950000,22.383333,45.608000,51.483333,13.437167
std,17.464249,15.578224,13.834126,31.406980,31.222597,6.362492,14.304557,16.118067,32.180190,31.589578,...,13.517628,15.508299,28.27733,29.529570,6.592724,15.082176,15.569518,28.013407,30.462810,7.455795
min,1.000000,1.000000,0.000000,0.120000,0.000000,0.910000,1.000000,0.000000,0.580000,1.000000,...,1.000000,0.000000,0.87000,0.000000,0.350000,0.000000,0.000000,0.500000,0.000000,0.020000
25%,15.750000,9.750000,15.000000,19.320000,29.750000,8.607500,7.750000,8.750000,20.055000,23.000000,...,14.000000,9.000000,21.49250,28.750000,5.830000,15.750000,6.750000,20.935000,25.750000,7.682500
50%,30.500000,24.000000,23.500000,41.880000,55.000000,13.440000,20.500000,27.000000,40.070000,50.000000,...,24.500000,22.000000,42.91000,56.000000,12.280000,25.500000,20.500000,42.125000,50.500000,13.855000
75%,45.250000,37.250000,35.250000,75.042500,84.750000,18.595000,32.250000,38.000000,77.875000,81.750000,...,36.000000,37.250000,63.36500,78.250000,17.887500,37.500000,34.250000,72.232500,74.250000,21.032500
max,60.000000,50.000000,50.000000,99.590000,99.000000,23.970000,49.000000,49.000000,98.650000,100.000000,...,50.000000,50.000000,96.26000,100.000000,22.450000,50.000000,50.000000,92.890000,97.000000,23.810000


In [10]:
''' finding the names of the top 10 students based on the average of final semester marks for all subjects.
    We can use this data later for evaluating the model.
'''


# Target columns
target_cols = ['c_final_exam_marks', 'java_final_exam_marks', 'python_final_exam_marks', 'cn_final_exam_marks', 'dsa_final_exam_marks']

# Calculate the average final semester marks for each student
data['avg_final_marks'] = data[target_cols].mean(axis=1)

# Sort the data based on the average final semester marks in descending order
sorted_data = data.sort_values(by='avg_final_marks', ascending=False)

# Print the names of the top 10 performers
print("Top 10 Performers (based on average final semester marks):")
top_10_students = sorted_data['student_name'].head(10)
for i, student in enumerate(top_10_students, start=1):
    print(f"{i}. {student}")

Top 10 Performers (based on average final semester marks):
1. Maryjo Grundy
2. Isis Kimpton
3. Rosabella Goldis
4. Indira Arnet
5. Sheelah Glenister
6. Iggie Gemeau
7. Barrett Pitfield
8. Beryle Buckbee
9. Keefer Lowis
10. Francoise Hillum


In [3]:
# splitting variables into features and targets
features = data.drop(['student_name', 'c_final_exam_marks', 'java_final_exam_marks', 'python_final_exam_marks', 'cn_final_exam_marks', 'dsa_final_exam_marks'], axis=1)
targets = data[['c_final_exam_marks', 'java_final_exam_marks', 'python_final_exam_marks', 'cn_final_exam_marks', 'dsa_final_exam_marks']]

In [11]:
features.head()

,student_id,c_internal_exam1_marks,c_internal_exam2_marks,c_attendance_percentage,c_preparation_time_daily_hours,java_internal_exam1_marks,java_internal_exam2_marks,java_attendance_percentage,java_preparation_time_daily_hours,python_internal_exam1_marks,...,python_attendance_percentage,python_preparation_time_daily_hours,cn_internal_exam1_marks,cn_internal_exam2_marks,cn_attendance_percentage,cn_preparation_time_daily_hours,dsa_internal_exam1_marks,dsa_internal_exam2_marks,dsa_attendance_percentage,dsa_preparation_time_daily_hours
0,1,2,18,23.46,11.68,35,29,34.43,18.94,16,...,88.14,13.38,33,38,78.81,3.37,11,38,21.86,16.76
1,2,26,25,6.42,7.94,31,21,2.45,15.11,32,...,4.15,20.07,2,49,26.49,12.34,0,28,77.70,13.11
2,3,17,32,55.67,16.15,6,20,54.37,23.43,44,...,38.24,9.49,50,48,52.05,18.47,32,8,11.64,23.69
3,4,7,11,73.16,9.23,28,30,69.54,20.73,48,...,0.20,0.99,25,39,62.62,19.42,16,14,86.84,22.30
4,5,45,6,23.29,10.80,1,1,37.48,5.48,36,...,93.88,10.90,22,8,91.70,10.92,25,33,62.14,8.54


In [12]:
targets.head()

,c_final_exam_marks,java_final_exam_marks,python_final_exam_marks,cn_final_exam_marks,dsa_final_exam_marks
0,14,40,100,25,88
1,84,85,88,1,91
2,29,96,17,58,89
3,20,58,43,50,70
4,97,91,48,61,95


In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)


In [5]:
# Create a Random Forest Regressor model for each subject in the data
c_model = RandomForestRegressor()
java_model = RandomForestRegressor()
python_model = RandomForestRegressor()
cn_model = RandomForestRegressor()
dsa_model = RandomForestRegressor()

In [13]:
c_model

RandomForestRegressor()

In [14]:
# Train the models for each subhject
c_model.fit(X_train, y_train['c_final_exam_marks'])
java_model.fit(X_train, y_train['java_final_exam_marks'])
python_model.fit(X_train, y_train['python_final_exam_marks'])
cn_model.fit(X_train, y_train['cn_final_exam_marks'])
dsa_model.fit(X_train, y_train['dsa_final_exam_marks'])

RandomForestRegressor()

In [15]:

# Evaluate the models on the testing set
c_preds = c_model.predict(X_test)
java_preds = java_model.predict(X_test)
python_preds = python_model.predict(X_test)
cn_preds = cn_model.predict(X_test)
dsa_preds = dsa_model.predict(X_test)


In [16]:
# predicted values
c_preds

array([43.64, 69.63, 48.73, 44.47, 85.  , 67.44, 48.34, 71.5 , 60.68,
       66.69, 55.56, 52.87])

In [17]:
# calculating RMSE values, lower values indicate better accuracy

c_rmse = mean_squared_error(y_test['c_final_exam_marks'], c_preds, squared=False)
java_rmse = mean_squared_error(y_test['java_final_exam_marks'], java_preds, squared=False)
python_rmse = mean_squared_error(y_test['python_final_exam_marks'], python_preds, squared=False)
cn_rmse = mean_squared_error(y_test['cn_final_exam_marks'], cn_preds, squared=False)
dsa_rmse = mean_squared_error(y_test['dsa_final_exam_marks'], dsa_preds, squared=False)


c:\Users\aniya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\aniya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\aniya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\aniya\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarnin

In [18]:
# printing the RMSE values
print(f"C RMSE: {c_rmse}")
print(f"Java RMSE: {java_rmse}")
print(f"Python RMSE: {python_rmse}")
print(f"CN RMSE: {cn_rmse}")
print(f"DSA RMSE: {dsa_rmse}")


C RMSE: 36.520186769328916
Java RMSE: 30.016390800138957
Python RMSE: 35.889372266266605
CN RMSE: 35.5959231045729
DSA RMSE: 36.70884316709894


Using 2 models
1. Fine tuned Random Forest Model
2. Gradient Boosting model

In [33]:
# Initialize models
rf_models = {}
gb_models = {}

In [34]:
# Grid search parameters
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}


In [35]:

# Train and tune Random Forest models
for subject in targets.columns:
    rf_model = RandomForestRegressor()
    grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='neg_root_mean_squared_error')
    grid_search.fit(X_train, y_train[subject])
    rf_models[subject] = grid_search.best_estimator_


In [36]:
# Train and tune Gradient Boosting models
for subject in targets.columns:
    gb_model = GradientBoostingRegressor()
    grid_search = GridSearchCV(gb_model, param_grid, cv=5, scoring='neg_root_mean_squared_error')
    grid_search.fit(X_train, y_train[subject])
    gb_models[subject] = grid_search.best_estimator_


In [37]:
# Evaluate models using cross-validation
rf_cv_scores = {subject: cross_val_score(model, X_train, y_train[subject], cv=5, scoring='neg_root_mean_squared_error').mean() for subject, model in rf_models.items()}
gb_cv_scores = {subject: cross_val_score(model, X_train, y_train[subject], cv=5, scoring='neg_root_mean_squared_error').mean() for subject, model in gb_models.items()}


In [38]:
# Print cross-validation scores
print("Random Forest Cross-Validation Scores:")
for subject, score in rf_cv_scores.items():
    print(f"{subject}: {score}")

print("\nGradient Boosting Cross-Validation Scores:")
for subject, score in gb_cv_scores.items():
    print(f"{subject}: {score}")

Random Forest Cross-Validation Scores:
c_final_exam_marks: -29.605832249422747
java_final_exam_marks: -35.83701458459332
python_final_exam_marks: -33.299802350409564
cn_final_exam_marks: -28.53069219169052
dsa_final_exam_marks: -32.60722087563651

Gradient Boosting Cross-Validation Scores:
c_final_exam_marks: -33.77486420441971
java_final_exam_marks: -32.890269449970404
python_final_exam_marks: -35.54389891474822
cn_final_exam_marks: -32.52378273848876
dsa_final_exam_marks: -32.504612593838495


In [39]:
# Select the best model for each subject based on cross-validation scores
best_rf_models = {subject: model for subject, model in rf_models.items() if rf_cv_scores[subject] == max(rf_cv_scores.values())}
best_gb_models = {subject: model for subject, model in gb_models.items() if gb_cv_scores[subject] == max(gb_cv_scores.values())}


In [40]:
# Make predictions for a specific student using the best models
student_name = "Maryjo Grundy"
student_row = data.loc[data['student_name'] == student_name]
student_features = features

predicted_marks_rf = {subject: model.predict(student_features)[0] for subject, model in best_rf_models.items()}
predicted_marks_gb = {subject: model.predict(student_features)[0] for subject, model in best_gb_models.items()}

In [42]:

# Get actual marks
actual_marks = {subject: student_row[subject].values[0] for subject in targets.columns}

# Print predicted and actual marks for the student using Random Forest
print("\nRandom Forest Predicted final semester marks for Maryjo Grundy:")
for subject, pred_mark in predicted_marks_rf.items():
    print(f"{subject.upper()}: {pred_mark:.2f} (Actual: {actual_marks[subject]})")


Random Forest Predicted final semester marks for Maryjo Grundy:
CN_FINAL_EXAM_MARKS: 60.59 (Actual: 61)


In [41]:
# Print predicted and actual marks for the student using Gradient Boosting
print("\nGradient Boosting Predicted final semester marks for Maryjo Grundy:")
for subject, pred_mark in predicted_marks_gb.items():
    print(f"{subject.upper()}: {pred_mark:.2f} (Actual: {actual_marks[subject]})")


Gradient Boosting Predicted final semester marks for Maryjo Grundy:
DSA_FINAL_EXAM_MARKS: 18.41 (Actual: 95)


Fine tuned Random Forest model provides better accuracy